In [1]:
from ctypes import *
import numpy as np
import time

In [2]:
import os
os.getcwd()

'/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll'

In [3]:
libc_FISTA_dll = CDLL("./fista_kernel.so")
FISTA_Kernel_S = libc_FISTA_dll.FISTA_S

In [4]:
libc_FISTA_blas = CDLL("./fista_cublas.so")
FISTA_blas_S = libc_FISTA_blas.FISTA_S

In [5]:
FISTA_Kernel_S.restype = None
FISTA_Kernel_S.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

In [6]:
FISTA_blas_S.restype = None
FISTA_blas_S.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

In [7]:
n = 500
p = 1000

import numpy as np
np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

In [8]:
## input
beta = np.zeros(p).astype(np.float32)
beta_cr = beta.ctypes.data_as(POINTER(c_float))
X_cr =  X.ctypes.data_as(POINTER(c_float))## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
y_cr = y.ctypes.data_as(POINTER(c_float))

In [9]:
## Parameters
max_iter = 5000
loss = np.zeros(max_iter).astype(np.float32)
loss_c = loss.ctypes.data_as(POINTER(c_float)) 
lam =  np.sqrt(2 * np.log(p) / n)
lam_c = c_float(lam)
L_c = c_float(10)
eta_c = c_float(2.0)
tol_c = c_float(1e-04)

iter_c = c_int(max_iter)
n_c = c_int(n)
p_c = c_int(p)
step_c = c_int(0)

In [10]:
tic = time.time()
FISTA_Kernel_S(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
toc = time.time()
print("Elapsed time: ", toc - tic, "secs")

Elapsed time:  1.9729347229003906 secs


In [11]:
step_c

c_int(1721)

In [12]:
beta[:10]

array([0.9855122 , 0.9981539 , 0.9732941 , 0.94141245, 0.9619718 ,
       0.93535745, 0.94177866, 1.0285132 , 0.8439329 , 0.92502004],
      dtype=float32)

In [13]:
## input
beta = np.zeros(p).astype(np.float32)
beta_cc = beta.ctypes.data_as(POINTER(c_float))
X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
X_cc = X_c.ctypes.data_as(POINTER(c_float))
#y_c = y.ravel(order = "F")
y_cc = y.ctypes.data_as(POINTER(c_float))

loss = np.zeros(5000).astype(np.float32)## max_iter size만큼
loss_c = loss.ctypes.data_as(POINTER(c_float)) 
lam =  np.sqrt(2 * np.log(p) / n)
lam_c = c_float(lam)
L_c = c_float(10)
eta_c = c_float(2.0)
tol_c = c_float(1e-04)

iter_c = c_int(5000)
n_c = c_int(n)
p_c = c_int(p)
step_c = c_int(0)

In [14]:
tic = time.time()
FISTA_blas_S(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
toc = time.time()
    
print("Elapsed time: ", toc - tic, "secs")

Elapsed time:  1.2909228801727295 secs


In [15]:
step_c

c_int(1721)

In [16]:
beta[:10]

array([0.9854982 , 0.99852866, 0.97280145, 0.9412902 , 0.961787  ,
       0.9346652 , 0.9422157 , 1.028061  , 0.8439065 , 0.925274  ],
      dtype=float32)

In [1]:
n = 500
p = 1000

import time
import numpy as np
from ctypes import *

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

libc_FISTA_dll = CDLL("./fista_kernel.so")
FISTA_Kernel_S = libc_FISTA_dll.FISTA_S

FISTA_Kernel_S.restype = None
FISTA_Kernel_S.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

niter = 11
comp_time_ker = np.zeros(niter)


for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float32)
    beta_cr = beta.ctypes.data_as(POINTER(c_float))
    X_cr =  X.ctypes.data_as(POINTER(c_float)) 
    y_cr = y.ctypes.data_as(POINTER(c_float))


    loss = np.zeros(5000).astype(np.float32)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_float)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_float(lam)
    L_c = c_float(10)
    eta_c = c_float(2.0)
    tol_c = c_float(1e-04)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_Kernel_S(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_ker[i] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_ker[i])
    



  1 -th iteration, Collapsed Time:  1.9186515808105469

  2 -th iteration, Collapsed Time:  1.746856689453125

  3 -th iteration, Collapsed Time:  1.7461161613464355

  4 -th iteration, Collapsed Time:  1.7446577548980713

  5 -th iteration, Collapsed Time:  1.7492587566375732

  6 -th iteration, Collapsed Time:  1.752138614654541

  7 -th iteration, Collapsed Time:  1.7510583400726318

  8 -th iteration, Collapsed Time:  1.7472164630889893

  9 -th iteration, Collapsed Time:  1.753765344619751

  10 -th iteration, Collapsed Time:  1.7499644756317139

  11 -th iteration, Collapsed Time:  1.7489697933197021


In [2]:
n = 500
p = 1000

import time
import numpy as np
from ctypes import *

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

libc_FISTA_blas = CDLL("./fista_cublas.so")
FISTA_blas_S = libc_FISTA_blas.FISTA_S

FISTA_blas_S.restype = None
FISTA_blas_S.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

niter = 11
comp_time_blas = np.zeros(niter)


for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float32)
    beta_cc = beta.ctypes.data_as(POINTER(c_float))
    X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    X_cc = X_c.ctypes.data_as(POINTER(c_float))
    y_cc = y.ctypes.data_as(POINTER(c_float))

    loss = np.zeros(5000).astype(np.float32)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_float)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_float(lam)
    L_c = c_float(10)
    eta_c = c_float(2.0)
    tol_c = c_float(1e-04)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_blas_S(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_blas[i] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_blas[i])
    



  1 -th iteration, Collapsed Time:  1.2808589935302734

  2 -th iteration, Collapsed Time:  0.9993188381195068

  3 -th iteration, Collapsed Time:  0.9991343021392822

  4 -th iteration, Collapsed Time:  0.999342679977417

  5 -th iteration, Collapsed Time:  0.9962754249572754

  6 -th iteration, Collapsed Time:  1.0015130043029785

  7 -th iteration, Collapsed Time:  0.9994418621063232

  8 -th iteration, Collapsed Time:  0.9991240501403809

  9 -th iteration, Collapsed Time:  0.9936964511871338

  10 -th iteration, Collapsed Time:  0.999852180480957

  11 -th iteration, Collapsed Time:  0.9994001388549805
